<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Pre-requisites" data-toc-modified-id="Pre-requisites-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Pre-requisites</a></span></li><li><span><a href="#Instructions" data-toc-modified-id="Instructions-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Instructions</a></span></li><li><span><a href="#Imports-and-Constants" data-toc-modified-id="Imports-and-Constants-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Imports and Constants</a></span></li><li><span><a href="#Validate-and-Split-Exported-TFRecords" data-toc-modified-id="Validate-and-Split-Exported-TFRecords-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Validate and Split Exported TFRecords</a></span></li><li><span><a href="#Calculate-Mean-and-Std-Dev-for-Each-Band" data-toc-modified-id="Calculate-Mean-and-Std-Dev-for-Each-Band-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Calculate Mean and Std-Dev for Each Band</a></span></li></ul></div>

## Pre-requisites

Go through the [`preprocessing/0_export_tfrecords.ipynb`](./0_export_tfrecords.ipynb) notebook.

Before running this notebook, you should have the following structure under the `data/` directory:

```
data/
    dhs_tfrecords_raw/
        angola_2011_00.tfrecord.gz
        ...
        zimbabwe_2015_XX.tfrecord.gz
    dhsnl_tfrecords_raw/
        angola_2010_00.tfrecord.gz
        ...
        zimbabwe_2016_XX.tfrecord.gz
    lsms_tfrecords_raw/
        ethiopia_2011_00.tfrecord.gz
        ...
        uganda_2013_XX.tfrecord.gz
```

## Instructions

This notebook processes the exported TFRecords as follows:
1. Verifies that the fields in the TFRecords match the original CSV files.
2. Splits each monolithic TFRecord file exported from Google Earth Engine into one file per record.

After running this notebook, you should have three new folders (`dhs_tfrecords`, `dhsnl_tfrecords`, and `lsms_tfrecords`) under `data/`:

```
data/
    dhs_tfrecords/
        angola_2011/
            00000.tfrecord.gz
            ...
            00229.tfrecord.gz
        ...
        zimbabwe_2015/
            00000.tfrecord.gz
            ...
            00399.tfrecord.gz
    dhsnl_tfrecords/
        angola_2010/
            00000.tfrecord.gz
            ...
            07734.tfrecord.gz
        zimbabwe_2016/
            00000.tfrecord.gz
            ...
            03584.tfrecord.gz
    lsms_tfrecords/
        ethiopia_2011/
            00000.tfrecord.gz
            ...
            00326.tfrecord.gz
        uganda_2013/
            00000.tfrecord.gz
            ...
            00164.tfrecord.gz
```

This notebook also calculates the mean and standard deviation of each band across each of the 3 datasets.

## Imports and Constants

In [1]:
%load_ext autoreload
%autoreload 2


# change directory to repo root, and verify
%cd '../'
!pwd

/atlas/u/erikrozi/rural-urban-Modelbias
/atlas/u/erikrozi/rural-urban-Modelbias


In [51]:
from __future__ import annotations

from collections.abc import Iterable
from glob import glob
from pprint import pprint
import os
from typing import Optional

import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm import tqdm
import re

from batchers import batcher, tfrecord_paths_utils
from preprocessing.helper import (
    analyze_tfrecord_batch,
    per_band_mean_std,
    print_analysis_results)

In [5]:
REQUIRED_BANDS = ['long','lat']
BUILDINGS_EXPORT_FOLDER ='/content/drive/MyDrive/dhs_geo'
BUILDINGS_PROCESSED_FOLDER='/content/drive/MyDrive/dhs_geo_processed'
#BUILDINGS_EXPORT_FOLDER = '/atlas/u/erikrozi/bias_mitigation/africa_poverty_clean/data/dhs_buildings_raw'
#BUILDINGS_PROCESSED_FOLDER = '/atlas/u/erikrozi/bias_mitigation/africa_poverty_clean/data/dhs_buildings'

## Validate and Split Exported TFRecords

In [54]:
def process_dataset(csv_path: str, input_dir: str, processed_dir: str) -> None:
    '''
    Args
    - csv_path: str, path to CSV of DHS or LSMS clusters
    - input_dir: str, path to TFRecords exported from Google Earth Engine
    - processed_dir: str, folder where to save processed TFRecords
    '''
    df = pd.read_csv(csv_path, float_precision='high', index_col=False)
    surveys = list(df.groupby(['country', 'year']).groups.keys())  # (country, year) tuples
    for country, year in surveys:
        country_year = f'{country}_{year}'
        print('Processing:', country_year)
        tfrecord_paths = glob(os.path.join(input_dir, country_year + '*')).sort(key=lambda f: int(re.sub('\D', '', f)))
        out_dir = os.path.join(processed_dir, country_year)
        os.makedirs(out_dir, exist_ok=True)
        subset_df = df[(df['country'] == country) & (df['year'] == year)].reset_index(drop=True)
        validate_and_split_tfrecords(
            tfrecord_paths=tfrecord_paths, out_dir=out_dir, df=subset_df)
def validate_and_split_tfrecords(
        tfrecord_paths: Iterable[str],
        out_dir: str,
        df: pd.DataFrame
        ) -> None:
    '''Validates and splits a list of exported TFRecord files (for a
    given country-year survey) into individual TFrecords, one per cluster.
    "Validating" a TFRecord comprises of 2 parts
    1) verifying that it contains the required bands
    2) verifying that its other features match the values from the dataset CSV
    Args
    - tfrecord_paths: list of str, paths to exported TFRecords files
    - out_dir: str, path to dir to save processed individual TFRecords
    - df: pd.DataFrame, index is sequential and starts at 0
    '''
    print(tfrecord_paths)
    # Create an iterator over the TFRecords file. The iterator yields
    # the binary representations of Example messages as strings.
    options = tf.io.TFRecordOptions(tf.compat.v1.io.TFRecordCompressionType.GZIP)
    # cast float64 => float32 and str => bytes
    for col in df.columns:
        if df[col].dtype == np.float64:
            df[col] = df[col].astype(np.float32)
        elif df[col].dtype == object:  # pandas uses 'object' type for str
            df[col] = df[col].astype(bytes)
    i = 0
    progbar = tqdm(total=len(df))
    print(df[df['lat']==-10.556163787841797])
    for tfrecord_path in tfrecord_paths:
        iterator = tf.compat.v1.io.tf_record_iterator(tfrecord_path, options=options)
        for record_str in iterator:
            # parse into an actual Example message
            ex = tf.train.Example.FromString(record_str)
            feature_map = ex.features.feature
            # verify required bands exist
            for band in REQUIRED_BANDS:
                assert band in feature_map, f'Band "{band}" not in record {i} of {tfrecord_path}'
            # compare feature map values against CSV values
            csv_feats = df.loc[i, :].to_dict()
            for col, val in csv_feats.items():
                ft_type = feature_map[col].WhichOneof('kind')
                ex_val = feature_map[col].__getattribute__(ft_type).value[0]
                assert val == ex_val, f'Expected {col}={val}, but found {ex_val} instead'
            # serialize to string and write to file
            out_path = os.path.join(out_dir, f'{i:05d}.tfrecord.gz')  # all surveys have < 1e6 clusters
            with tf.io.TFRecordWriter(out_path, options=options) as writer:
                writer.write(ex.SerializeToString())
            i += 1
            progbar.update(1)
    progbar.close()

In [53]:
process_dataset(
    csv_path='/atlas/u/erikrozi/bias_mitigation/africa_poverty_clean/data/dhs_clusters.csv',
    input_dir=BUILDINGS_EXPORT_FOLDER,
    processed_dir=BUILDINGS_PROCESSED_FOLDER)

Processing: angola_2011
None
['/atlas/u/erikrozi/bias_mitigation/africa_poverty_clean/data/dhs_buildings_raw/angola_2011_00.tfrecord.gz', '/atlas/u/erikrozi/bias_mitigation/africa_poverty_clean/data/dhs_buildings_raw/angola_2011_01.tfrecord.gz', '/atlas/u/erikrozi/bias_mitigation/africa_poverty_clean/data/dhs_buildings_raw/angola_2011_02.tfrecord.gz', '/atlas/u/erikrozi/bias_mitigation/africa_poverty_clean/data/dhs_buildings_raw/angola_2011_03.tfrecord.gz', '/atlas/u/erikrozi/bias_mitigation/africa_poverty_clean/data/dhs_buildings_raw/angola_2011_04.tfrecord.gz', '/atlas/u/erikrozi/bias_mitigation/africa_poverty_clean/data/dhs_buildings_raw/angola_2011_05.tfrecord.gz', '/atlas/u/erikrozi/bias_mitigation/africa_poverty_clean/data/dhs_buildings_raw/angola_2011_06.tfrecord.gz', '/atlas/u/erikrozi/bias_mitigation/africa_poverty_clean/data/dhs_buildings_raw/angola_2011_07.tfrecord.gz', '/atlas/u/erikrozi/bias_mitigation/africa_poverty_clean/data/dhs_buildings_raw/angola_2011_08.tfrecord.gz'

  0%|          | 1/230 [00:00<00:43,  5.25it/s]

      country  year        lat        lon      GID_1        GID_2  \
70  b'angola'  2011 -10.556164  22.039558  b'AGO.13'  b'AGO.13.3'   

    wealthpooled  households  urban_rural  
70     -1.128011          37            0  


100%|██████████| 230/230 [00:06<00:00, 34.35it/s]


Processing: angola_2015
None
['/atlas/u/erikrozi/bias_mitigation/africa_poverty_clean/data/dhs_buildings_raw/angola_2015_00.tfrecord.gz']


...

100%|██████████| 393/393 [00:03<00:00, 117.85it/s]


Processing: zimbabwe_2015
None
['/atlas/u/erikrozi/bias_mitigation/africa_poverty_clean/data/dhs_buildings_raw/zimbabwe_2015_00.tfrecord.gz']


  0%|          | 0/400 [00:00<?, ?it/s]

Empty DataFrame
Columns: [country, year, lat, lon, GID_1, GID_2, wealthpooled, households, urban_rural]
Index: []


100%|██████████| 400/400 [00:03<00:00, 123.13it/s]
